In [42]:
import pytz
from datetime import datetime, date, timedelta
import pandas as pd
import altair as alt
import altair_latimes as lat

In [43]:
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

In [44]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Import

In [45]:
df = pd.read_csv(
    "../data/raw/reservoirs/major-reservoir-scrape-latest.csv", 
    parse_dates=["DATE TIME", "OBS DATE"]
)

In [46]:
hist_df = pd.read_csv(
    "../data/metadata/reservoirs-historical-averages.csv"
)

In [47]:
details_df = pd.read_csv(
    "../data/metadata/reservoirs-details.csv"
)

### Clean

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6496 entries, 0 to 6495
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   STATION_ID     6496 non-null   object        
 1   DURATION       6496 non-null   object        
 2   SENSOR_NUMBER  6496 non-null   int64         
 3   SENSOR_TYPE    6496 non-null   object        
 4   DATE TIME      6496 non-null   datetime64[ns]
 5   OBS DATE       6496 non-null   datetime64[ns]
 6   VALUE          6496 non-null   object        
 7   DATA_FLAG      6496 non-null   object        
 8   UNITS          6496 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 456.9+ KB


In [49]:
df.columns = df.columns.str.lower()

In [50]:
df.columns = df.columns.str.replace(" ","_")

In [51]:
df = df.rename(columns={
    "station_id": "reservoir_id",
    "date_time": "date",
    "value": "storage_af"
})

In [52]:
df#[df["reservoir_id"] == "CLA"]

,reservoir_id,duration,sensor_number,sensor_type,date,obs_date,storage_af,data_flag,units
0,DRE,D,15,STORAGE,2022-05-17,2022-05-18,5705,,AF
1,DRE,D,15,STORAGE,2022-05-18,2022-05-19,5657,,AF
2,DRE,D,15,STORAGE,2022-05-19,2022-05-20,5645,,AF
3,DRE,D,15,STORAGE,2022-05-20,2022-05-21,5613,,AF
4,DRE,D,15,STORAGE,2022-05-21,2022-05-22,5578,,AF
...,...,...,...,...,...,...,...,...,...
6491,SLW,D,15,STORAGE,2022-06-13,2022-06-14,67983,,AF
6492,SLW,D,15,STORAGE,2022-06-14,2022-06-15,68020,,AF
6493,SLW,D,15,STORAGE,2022-06-15,2022-06-16,68039,,AF
6494,SLW,D,15,STORAGE,2022-06-16,2022-06-17,68076,,AF


In [53]:
df["storage_af"] = df["storage_af"].str.replace("---","")

In [54]:
df["storage_af"] = pd.to_numeric(df["storage_af"])

In [55]:
#df = df.dropna(subset="storage_af")

### Check out latest data

In [56]:
tz = pytz.timezone("America/Los_Angeles")

In [57]:
today = datetime.now(tz).date()
today

datetime.date(2022, 6, 17)

In [58]:
yesterday = (today - pd.DateOffset(days=1)).date()
yesterday

datetime.date(2022, 6, 16)

In [59]:
latest_df = df[df.date == pd.to_datetime(yesterday)].copy()

In [60]:
len(latest_df)

203

In [61]:
latest_df.storage_af.sum()

18821910.0

### Merge historical average

Create month column for merging

In [62]:
latest_df["month"] = pd.DatetimeIndex(latest_df.date).month

In [63]:
hist_df["month"] = pd.to_datetime(hist_df["month"], format='%B').dt.month

In [64]:
merge_historical = pd.merge(
    latest_df[["reservoir_id","date","month","storage_af","units"]],
    hist_df[["reservoir_id", "month", "average_storage_value", "average_storage_unit"]],
    how="left",
    on=["reservoir_id","month"]
)

In [65]:
merge_details = pd.merge(
    merge_historical,
    details_df[["reservoir_id", "lake_name", "capacity_value", "capacity_unit"]],
    how="left",
    on=["reservoir_id"]               
)

### Calculate percentages

In [66]:
merge_details["current_level_pct_of_total"] = merge_details["storage_af"] / merge_details["capacity_value"]

In [67]:
merge_details["average_level_pct_of_total"] = merge_details["average_storage_value"] / merge_details["capacity_value"]

In [68]:
merge_details["current_level_pct_of_avg"] = merge_details["storage_af"] / merge_details["average_storage_value"]

### Filter to just "major" reservoirs

In [69]:
# src: https://cdec.water.ca.gov/resapp/RescondMain
major_reservoirs_list = [
    'SHA',
    'ORO',
    'BUL',
    'FOL',
    'CMN',
    'NML',
    'DNP',
    'EXC',
    'CLE',
    'WRS',
    'SNL',
    'CCH',
    'CAS',
    'CSI',
    'DMV',
    'MIL',
    'PNF',
]

In [70]:
major_reservoirs = merge_details[merge_details.reservoir_id.isin(major_reservoirs_list)]

Drop missing values

In [71]:
drop_na = major_reservoirs.dropna(subset=["storage_af","average_storage_value"])

In [72]:
len(drop_na)

16

In [73]:
melt=pd.melt(drop_na, id_vars="reservoir_id", value_vars=["storage_af", "average_storage_value", "capacity_value"])

In [74]:
bar_order = {
    "storage_af": 1, 
    "average_storage_value": 2, 
    "capacity_value":3            
}

In [76]:
melt["bar_order"] = melt.variable.map(bar_order)

In [77]:
domain = ["capacity_value", "average_storage_value", "storage_af", ]
range_ = ['#ddd', '#83c6e0', '#1281aa', ]

alt.Chart(melt).mark_bar().encode(
    x=alt.X('reservoir_id', axis=alt.Axis(labels=False)),
    y=alt.Y('value',stack="normalize"),
    order="bar_order",
    color=alt.Color(
        'variable', 
        scale=alt.Scale(domain=domain, range=range_), 
        #sort='descending'
    ),
    tooltip=["reservoir_id"]
).properties(title="California's largest reservoirs")

alt.Chart(...)

In [78]:
domain = ["capacity_value", "average_storage_value", "storage_af", ]
range_ = ['#ddd', '#83c6e0', '#1281aa', ]

alt.Chart(melt[melt.variable != "capacity_value"]).mark_bar().encode(
    x=alt.X('reservoir_id', axis=alt.Axis(labels=False)),
    y=alt.Y('value',stack="normalize"),
    order="bar_order",
    color=alt.Color(
        'variable', 
        scale=alt.Scale(domain=domain, range=range_), 
        #sort='descending'
    ),
    tooltip=["reservoir_id"]
).properties(title="California's largest reservoirs")

alt.Chart(...)

In [79]:
domain = ["capacity_value", "average_storage_value", "storage_af", ]
range_ = ['#ddd', '#83c6e0', '#1281aa', ]

alt.Chart(melt[melt.reservoir_id=="ORO"]).mark_bar().encode(
    x=alt.X('reservoir_id', axis=alt.Axis(labels=False)),
    y=alt.Y('value',stack="normalize"),
    order="bar_order",
    color=alt.Color(
        'variable', 
        scale=alt.Scale(domain=domain, range=range_), 
        #sort='descending'
    ),
    tooltip=["reservoir_id"]
).properties(title="Lake Oroville", width=150)

alt.Chart(...)

### Export

Assertion tests to make sure no data comes through with mixed units

In [80]:
assert len(drop_na.units.unique()) == 1

In [81]:
assert len(drop_na.average_storage_unit.unique()) == 1

In [82]:
assert len(drop_na.capacity_unit.unique()) == 1

Trim and rename for export

In [83]:
export_df = drop_na[
    ['reservoir_id','lake_name', 'date', 'storage_af', 'average_storage_value', 'capacity_value', 'current_level_pct_of_total',
       'average_level_pct_of_total', 'current_level_pct_of_avg' ]
].rename(columns={
    'lake_name': 'reservoir_name',
    'average_storage_value': 'historical_average',
    'capacity_value': 'total_capacity'
})

In [84]:
export_df.to_csv("../data/processed/reservoirs/reservoirs-latest.csv", index=False)

In [85]:
export_df.to_csv(f"../data/processed/reservoirs/reservoirs-{today}.csv", index=False)

---

### Merge geo data

In [108]:
metadata_df = pd.read_csv("../data/metadata/reservoirs-metadata-details.csv")

In [111]:
metadata_df["lat"] = metadata_df["lat"].str.replace("°","")
metadata_df["lon"] = metadata_df["lon"].str.replace("°","")

In [147]:
merge_coords = pd.merge(
    drop_na,
    metadata_df[["id","lat","lon"]],
    how="left",
    left_on=["reservoir_id"],
    right_on=["id"]
).drop(["id","month","units","average_storage_unit","capacity_unit"], axis=1)

In [148]:
merge_coords.lake_name = merge_coords.lake_name.str.strip()

In [151]:
merge_coords#[merge_coords.lake_name=="Lake Shasta"]

,reservoir_id,date,storage_af,average_storage_value,lake_name,capacity_value,current_level_pct_of_total,average_level_pct_of_total,current_level_pct_of_avg,lat,lon
0,CLE,2022-06-16,730476.000,1882223.000,Trinity Lake,2447650.000,0.298,0.769,0.388,40.801000,-122.762000
1,WRS,2022-06-16,134904.000,228117.000,Warm Springs Reservoir,381000.000,0.354,0.599,0.591,38.723000,-123.010000
2,CCH,2022-06-16,83515.000,144299.000,Cachuma Lake,193305.000,0.432,0.746,0.579,34.583000,-119.980000
3,CSI,2022-06-16,79021.000,192450.000,Lake Casitas,254000.000,0.311,0.758,0.411,34.373000,-119.332000
4,CAS,2022-06-16,124353.000,280108.000,Castaic Lake,325000.000,0.383,0.862,0.444,34.515200,-118.610100
5,SHA,2022-06-16,1805176.000,3539810.000,Lake Shasta,4552000.000,0.397,0.778,0.510,40.718000,-122.420000
6,ORO,2022-06-16,1837379.000,2687874.000,Lake Oroville,3537577.000,0.519,0.760,0.684,39.540000,-121.493000
7,BUL,2022-06-16,866610.000,831313.000,Bullards Bar Reservoir,966000.000,0.897,0.861,1.042,39.393000,-121.140000
8,FOL,2022-06-16,861301.000,756266.000,Folsom Lake,977000.000,0.882,0.774,1.139,38.683000,-121.183000
9,CMN,2022-06-16,216790.000,305377.000,Camanche Reservoir,417120.000,0.520,0.732,0.710,38.225000,-121.021000


In [150]:
merge_coords.to_csv("../data/processed/reservoirs/reservoirs-with-points.csv", index=False)